In [ ]:
import zarrita
import os

In [ ]:
root_path = "s3://carbonplan-benchmarks/data"
orig_stores = [
    # "nex-gddp-cmip6/monthly/CMIP6_ensemble_median/tasmax/tasmax_month_ensemble-median_historical.zarr",
    "NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn"
]
fp = os.path.join(root_path, orig_stores[0])

In [ ]:
targets = {
    1: "pyramids-v2-3857-0",
    # 5: "pyramids-v3-3857-1",
    # 10: "pyramids-v3-3857-2",
    # 20: "pyramids-v3-3857-3",
}

groups = ['0', '1', '2', '3']
arrays = ['tasmax', 'time', 'x', 'y']

In [ ]:
def convert_group_to_v3(*, group: str, source_store: str, target_store: str):
    source_group = zarrita.GroupV2.open(source_store / group)
    attrs = source_group.attributes
    zarrita.Group.create(target_store / group, attributes=attrs)


def convert_array_to_v3(*, group: str, array: str, source_store: str, target_store: str):
    source_array = zarrita.ArrayV2.open(source_store / group / array)
    data = source_array[:]
    attrs = source_array.attributes
    chunks = source_array.metadata.chunks
    target_array = zarrita.Array.create(
        target_store / group / array,
        shape=data.shape,
        dtype=data.dtype,
        chunk_shape=chunks,
        attributes=attrs,
        codecs=[zarrita.codecs.endian_codec(), zarrita.codecs.gzip_codec()],
    )
    target_array[:] = data

In [ ]:
def convert_store_to_v3(*, source_store, target_store):
    for group in groups:
        convert_group_to_v3(group=group, source_store=source_store, target_store=target_store)
        for array in arrays:
            convert_array_to_v3(
                group=group, array=array, source_store=source_store, target_store=target_store
            )

In [ ]:
for store in orig_stores:
    for target_mb, path in targets.items():
        orig = os.path.join(root_path, store, path)
        target = orig.replace("v2", "v3")
        source_store = zarrita.RemoteStore(orig)
        target_store = zarrita.RemoteStore(target)
        convert_store_to_v3(source_store=source_store, target_store=target_store)